<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [1]:
!pip3 install twilio

In [2]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [3]:
query = 'Lima'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=ba8ddd3daa6d497e8dd161030240802&q=Lima&days=1&aqi=no&alerts=no'

In [4]:
response = requests.get(url_clima).json()

In [5]:
response

{'location': {'name': 'Lima',
  'region': 'Lima',
  'country': 'Peru',
  'lat': -12.05,
  'lon': -77.05,
  'tz_id': 'America/Lima',
  'localtime_epoch': 1707434242,
  'localtime': '2024-02-08 18:17'},
 'current': {'last_updated_epoch': 1707434100,
  'last_updated': '2024-02-08 18:15',
  'temp_c': 25.9,
  'temp_f': 78.6,
  'is_day': 1,
  'condition': {'text': 'Partly Cloudy ',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 7.8,
  'wind_kph': 12.6,
  'wind_degree': 186,
  'wind_dir': 'S',
  'pressure_mb': 1010.0,
  'pressure_in': 29.83,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 73,
  'cloud': 31,
  'feelslike_c': 27.8,
  'feelslike_f': 82.1,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 10.0,
  'gust_mph': 9.0,
  'gust_kph': 14.5},
 'forecast': {'forecastday': [{'date': '2024-02-08',
    'date_epoch': 1707350400,
    'day': {'maxtemp_c': 27.2,
     'maxtemp_f': 80.9,
     'mintemp_c': 22.8,
     'mintemp_f': 73.1,
     'avgtemp_c'

In [6]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [7]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [8]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [9]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1707368400,
 'time': '2024-02-08 00:00',
 'temp_c': 24.3,
 'temp_f': 75.8,
 'is_day': 0,
 'condition': {'text': 'Clear ',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
  'code': 1000},
 'wind_mph': 7.6,
 'wind_kph': 12.2,
 'wind_degree': 178,
 'wind_dir': 'S',
 'pressure_mb': 1012.0,
 'pressure_in': 29.9,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 80,
 'cloud': 17,
 'feelslike_c': 26.3,
 'feelslike_f': 79.4,
 'windchill_c': 24.4,
 'windchill_f': 75.8,
 'heatindex_c': 26.3,
 'heatindex_f': 79.4,
 'dewpoint_c': 20.8,
 'dewpoint_f': 69.4,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 9.2,
 'gust_kph': 14.8,
 'uv': 1.0,
 'short_rad': 0.0,
 'diff_rad': 0.0}

In [10]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2024-02-08'

In [11]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [13]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [14]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [15]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


In [16]:
datos[0]

('2024-02-08', 0, 'Clear ', 24.3, 0, 0)

In [17]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-02-08,0,Clear,24.3,0,0
1,2024-02-08,1,Partly Cloudy,24.0,0,0
2,2024-02-08,2,Partly Cloudy,23.8,0,0
3,2024-02-08,3,Partly Cloudy,23.6,0,0
4,2024-02-08,4,Partly Cloudy,23.3,0,0
5,2024-02-08,5,Partly Cloudy,23.1,0,0
6,2024-02-08,6,Cloudy,22.8,0,0
7,2024-02-08,7,Cloudy,23.4,0,0
8,2024-02-08,8,Partly Cloudy,24.0,0,0
9,2024-02-08,9,Partly Cloudy,24.7,0,0


In [18]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)
# Cambio adicional

In [19]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
22,2024-02-08,22,Patchy rain nearby,24.7,1,100


In [20]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [21]:
df['Fecha'][0]

'2024-02-08'

In [22]:
df_rain

,Condicion
Hora,


In [23]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2024-02-08 en Lima es : \n\n\n Empty DataFrame\nColumns: [Condicion]\nIndex: []'

In [24]:
PHONE_NUMBER

'+1 251 357 9583'

# Mensaje SMS desde Twilio

In [25]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+51992634890'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMdbc63ce200942c775c21a18364cbcaaa


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />